In [ ]:
import sys
from PyQt5.QtGui import *
from PyQt5.QtCore import *
#from PyQt5.QtWebKit import *
from lxml import html
import time
import random
#import cPickle as pickle
import bs4
import urllib
import urllib.request
import csv
import os
#from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtWebEngineWidgets import QWebEnginePage, QWebEngineView
import pickle
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl

def write_row_to_csv(row):
    with open('d:\data\data.csv','w',encoding='utf8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)

#Take this class for granted.Just use result of rendering.
# class Render(QWebEngineView):
#     def __init__(self, url):
#         if QApplication.instance():
#             self.app = QApplication.instance()
#         else:
#             self.app = QApplication(sys.argv)
#         QWebEngineView.__init__(self)
#         self.loadFinished.connect(self._loadFinished)
#         self.load(QUrl(url))
#         self.app.exec_()

#     def _loadFinished(self, result):
#         #self.frame = self.mainFrame()
#         self.page().toHtml(self.callable)
#         self.app.quit()

class Render(QWebEngineView):               # 子类Render继承父类QWebEngineView
    def __init__(self, url):
        self.html = ''
        self.app = QApplication(sys.argv)
        QWebEngineView.__init__(self)       # 子类构造函数继承父类，这种写法python2和3通用，还可以是super().__init__()
        self.loadFinished.connect(self._loadFinished)
        self.load(QUrl(url))
        self.app.exec_()

    def _loadFinished(self):
        self.page().toHtml(self.callable)

    def callable(self, data):
        self.html = data
        self.app.quit()

base_url = 'https://www.designhill.com/logo-design/contest/agriculture-logo-design-required-141296'
page_numbers = [''] + ['?page=' + str(x) for x in range(5)]
contest_links = []
gather_contest_links = False

if gather_contest_links:
    for page in page_numbers:
        url = base_url + page
        r = Render(url)
        result = r.html
        #This step is important.Converting QString to Ascii for lxml to process
        links = html.fromstring(str(result.encode('utf-8'))).find_class('allEntriesLink')
        for link in links:
            for i, a in enumerate(html.iterlinks(link)):
                if i == 0:
                    #print(a[2])
                    contest_links.append(a[2])

        time.sleep(10 + (random.random() * 10))

    pickle.dump(contest_links, open("d:\\contest_links.pickle", "wb"))

if not gather_contest_links:
    contest_links = pickle.load(open("d:\\contest_links.pickle", "rb"))

if "completed_links.pickle" in os.listdir('.'):
    completed_links = pickle.load(open("d:\\completed_links.pickle", "rb"))
else:
    completed_links = []

for link in contest_links:
    if link in completed_links:
        continue
    #print(link)
    r = Render(link)
    result = r.html
    html_string = str(result.encode('utf-8'))
    soup = bs4.BeautifulSoup(html_string, 'html.parser')
    entries = soup.find_all('div', class_='entries-box')
    name = link.split('/')[-1]
    for entry in entries:
        ribbons = entry.find_all('div', class_='ribbon_text')
        if len(ribbons) > 0:
            if 'winner' in ribbons[0].text.lower():
                winner = True
            else:
                winner = False
        else:
            winner = False

        img_link = entry.find_all('div', class_='personalDesImg')
        if len(img_link) > 0:
            img_link = img_link[0].find_all('img')[0]['src']
        else:
            continue

        stars = int(entry.find_all('span', class_='star-rating')[0]['data-rating'])
        author = entry.find_all('span', class_='author-name')[0].string
        urllib.request.urlretrieve(img_link, 'C:\\pics/' + img_link.split('/')[-1].split('?')[0])
        number = entry.find_all('div', class_='color-code')[0].string.strip('#')
        write_row_to_csv([name, number, author, stars, img_link.split('/')[-1].split('?')[0]])
        time.sleep(5 + (random.random() * 5))
    time.sleep(10 + (random.random() * 10))
    completed_links.append(link)
    pickle.dump(completed_links, open("d:\\completed_links.pickle", "wb"))